In [1]:
import pandas as pd
import numpy as np

import warnings 
warnings.simplefilter('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from pyspark.ml.classification import GBTClassifier
import findspark
findspark.init()
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
from pyspark.sql import Row
import collections
from pyspark import SparkConf,SparkContext
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
df = pd.read_csv('churn.csv')
df.shape

(900, 8)

In [4]:
del df['Unnamed: 0']
df.head()

,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn
0,Cameron Williams,42.0,11066.80,0,7.22,8.0,1
1,Kevin Mueller,41.0,11916.22,0,6.50,11.0,1
2,Eric Lozano,38.0,12884.75,0,6.67,12.0,1
3,Phillip White,42.0,8010.76,0,6.71,10.0,1
4,Cynthia Norton,37.0,9191.58,0,5.56,9.0,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Names            900 non-null    object 
 1   Age              900 non-null    float64
 2   Total_Purchase   900 non-null    float64
 3   Account_Manager  900 non-null    int64  
 4   Years            900 non-null    float64
 5   Num_Sites        900 non-null    float64
 6   Churn            900 non-null    int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 49.3+ KB


In [6]:
df["Churn"].value_counts()

0    750
1    150
Name: Churn, dtype: int64

In [7]:
df["Account_Manager"].value_counts()

0    467
1    433
Name: Account_Manager, dtype: int64

In [8]:
del df["Names"]

In [9]:
from __future__ import print_function

df=SparkSession.builder.getOrCreate()

22/08/26 22:27:58 WARN Utils: Your hostname, Gonen.local resolves to a loopback address: 127.0.0.1; using 192.168.10.94 instead (on interface en0)
22/08/26 22:27:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/26 22:27:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
inputlines=df = df.read.options(header = True, inferSchema = True).csv("churn.csv")

In [11]:
df=inputlines.rdd.map(lambda x: x.split(","))

In [12]:
df1 = inputlines
df1

DataFrame[_c0: int, Names: string, Age: double, Total_Purchase: double, Account_Manager: int, Years: double, Num_Sites: double, Churn: int]

In [13]:
from  pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=['Age','Total_Purchase','Account_Manager','Years','Num_Sites'], outputCol="features")
df1= vecAssembler.transform(df1)
df1

DataFrame[_c0: int, Names: string, Age: double, Total_Purchase: double, Account_Manager: int, Years: double, Num_Sites: double, Churn: int, features: vector]

In [14]:
df1 = df1.drop("index", 'Age','Total_Purchase','Account_Manager','Years','Num_Sites')
df1

DataFrame[_c0: int, Names: string, Churn: int, features: vector]

In [15]:
df1 = df1.drop("index", 'Age','Total_Purchase','Account_Manager','Years','Num_Sites')
df1 = df1.withColumnRenamed("Churn", "label")


In [16]:
trainTest=df1.randomSplit([0.5,0.5])
train=trainTest[0]
test=trainTest[1]

In [17]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [18]:
gbt=GBTClassifier(featuresCol='features',maxIter=10)

In [19]:
gbt = gbt.fit(train)

22/08/26 22:28:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Names, Age, Total_Purchase, Account_Manager, Years, Num_Sites, Churn
 Schema: _c0, Names, Age, Total_Purchase, Account_Manager, Years, Num_Sites, Churn
Expected: _c0 but found: 
CSV file: file:///Users/enestunahan/Documents/Data%20Science/Projects/Big%20Data%20Machine%20Learning%20Classification/churn.csv



[Stage 2:>                                                          (0 + 1) / 1]



22/08/26 22:28:12 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Names, Age, Total_Purchase, Account_Manager, Years, Num_Sites, Churn
 Schema: _c0, Names, Age, Total_Purchase, Account_Manager, Years, Num_Sites, Churn
Expected: _c0 but found: 
CSV file: file:///Users/enestunahan/Documents/Data%20Science/Projects/Big%20Data%20Machine%20Learning%20Classification/churn.csv
22/08/26 22:28:12 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Names, Age, Total_Purchase, Account_Manager, Years, Num_Sites, Churn
 Schema: _c0, Names, Age, Total_Purchase, Account_Manager, Years, Num_Sites, Churn
Expected: _c0 but found: 
CSV file: file:///Users/enestunahan/Documents/Data%20Science/Projects/Big%20Data%20Machine%20Learning%20Classification/churn.csv
22/08/26 22:28:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Names, Age, Total_Purchase, Account_Manager, Years, Num_Sites, Churn
 Schema: _c0, Names, Age, Total_

In [20]:
predictions = gbt.transform(test)
predictions.show()

22/08/26 22:28:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Names, Age, Total_Purchase, Account_Manager, Years, Num_Sites, Churn
 Schema: _c0, Names, Age, Total_Purchase, Account_Manager, Years, Num_Sites, Churn
Expected: _c0 but found: 
CSV file: file:///Users/enestunahan/Documents/Data%20Science/Projects/Big%20Data%20Machine%20Learning%20Classification/churn.csv
22/08/26 22:28:18 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/08/26 22:28:18 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
+---+------------------+-----+--------------------+--------------------+--------------------+----------+
|_c0|             Names|label|            features|       rawPrediction|         probability|prediction|
+---+------------------+-----+--------------------+--------------------+--------------------+----------+
|  0|  Cameron Williams|    1|[42.0,1

In [21]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)

22/08/26 22:28:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Names, Age, Total_Purchase, Account_Manager, Years, Num_Sites, Churn
 Schema: _c0, Names, Age, Total_Purchase, Account_Manager, Years, Num_Sites, Churn
Expected: _c0 but found: 
CSV file: file:///Users/enestunahan/Documents/Data%20Science/Projects/Big%20Data%20Machine%20Learning%20Classification/churn.csv
Accuracy: 0.8699551569506726
